In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE104006"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE104006"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE104006.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE104006.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE104006.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"miRNA and gene expression profiling in human thyroid carcinomas and non-neoplastic thyroids"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease: Thyroid_carcinoma', 'disease: Non-neoplastic_thyroid'], 1: ['histology: PDTC', 'histology: PDTC+ATC', 'histology: PTC', 'histology: Non-neoplastic_thyroid', 'histology: PDTC+PTC', 'histology: PTC_lymph_node_metastasis', 'histology: PTC+PDTC'], 2: ['age: 74', 'age: 67', 'age: 72', 'age: 38', 'age: 50', 'age: 41', 'age: 51', 'age: 73', 'age: 52', 'age: 48', 'age: 59', 'age: 58', 'age: 39', 'age: 37', 'age: 33', 'age: 36', 'age: 70', 'age: 26', 'age: 46', 'age: 57', 'age: 44', 'age: 35', 'age: 42', 'age: 61', 'age: 49'], 3: ['Sex: M', 'Sex: F']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Callable, Optional, Dict, Any
import json

# 1. Gene Expression Data Availability
# Based on the title and summary, this dataset appears to contain gene expression data
is_gene_available = True

# 2.1 Data Availability
# For trait, key 0 contains disease status (Thyroid_carcinoma vs Non-neoplastic_thyroid)
trait_row = 0
# For age, key 2 contains age information
age_row = 2
# For gender, key 3 contains sex information
gender_row = 3

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary format: 1 for Thyroid_carcinoma, 0 for Non-neoplastic_thyroid."""
    if isinstance(value, str):
        # Extract the value after the colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if value.lower() == 'thyroid_carcinoma':
            return 1
        elif value.lower() == 'non-neoplastic_thyroid':
            return 0
    return None

def convert_age(value):
    """Convert age value to continuous format."""
    if isinstance(value, str):
        # Extract the value after the colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)
        except ValueError:
            pass
    return None

def convert_gender(value):
    """Convert gender value to binary format: 1 for male, 0 for female."""
    if isinstance(value, str):
        # Extract the value after the colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if value.upper() == 'M':
            return 1
        elif value.upper() == 'F':
            return 0
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# For this step, we'll check if clinical data exists before processing
if trait_row is not None:
    clinical_file_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    if os.path.exists(clinical_file_path):
        try:
            # Load the clinical data
            clinical_data = pd.read_csv(clinical_file_path)
            
            # Use the function to extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the dataframe
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Save the processed clinical data
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        except Exception as e:
            print(f"Error processing clinical data: {e}")
    else:
        print(f"Clinical data file not found at {clinical_file_path}")
        print("Skipping clinical feature extraction.")
else:
    print("No trait data available, skipping clinical feature extraction.")


Clinical data file not found at ../../input/GEO/Thyroid_Cancer/GSE104006/clinical_data.csv
Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE104006/GSE104006_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE104006/GSE104006-GPL14951_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (29377, 34)
First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286', 'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315']


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the output
# These identifiers like 'hsa-let-7a-3p', 'hsa-let-7b-5p' appear to be microRNA identifiers
# The 'hsa-' prefix indicates human (Homo sapiens) microRNAs
# These are standard microRNA nomenclature, not gene symbols like BRCA1 or TP53
# They don't require mapping to gene symbols as they're already in a standard format

requires_gene_mapping = False